In [1]:
# The main ideas over here is that we could take recordings of different words one by one but with phrases
# while being pronounced within sentences. Each word should be pronouced distinctly. 

# The way we get each individual word from the recording is by croping the noise part that is to say cutting
# the less amplitude part. So we need a very finesly noiseless audio.

# All we need to reassemble the words with proper spaceing gap, comma gap and period gap.

# That's it..!!

In [2]:
# Let's make some sentences.
# 1. d m i faeneans me ap ka swagat hae .
# 2. lon sambandhit jankari ke liye ek dabaen .
# 3. hamare grahak seva pratinidhi se baat karne ke liye teen dabaen .


In [3]:
# Importing necessary packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
import tensorflow as tf
import librosa
from IPython.display import Audio
import random
import soundfile as sf
import pyrubberband


2022-03-11 14:50:07.866637: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-11 14:50:07.866666: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
# Now let's find all the unique words for it. 
texts = '''
d m i faeneans me _ ap ka swagat hae . 
lon sambandhit jankari ke liye ek dabaen . 
hamare grahak seva pratinidhi se baat karne ke liye , do dabaen . 
'''

print(texts, end='\n\n')

# Removing the new lines from the text corpus.
texts_clean = texts.replace('\n', '')
print(texts_clean, end='\n\n')

# Converting  the texts to array.
text_arr = texts_clean.split(' ')
print(text_arr, end='\n\n')

# Converting to set to get the unique.
text_set = set(text_arr)
print(text_set, end='\n\n')

print(len(text_set))



d m i faeneans me _ ap ka swagat hae . 
lon sambandhit jankari ke liye ek dabaen . 
hamare grahak seva pratinidhi se baat karne ke liye , do dabaen . 


d m i faeneans me _ ap ka swagat hae . lon sambandhit jankari ke liye ek dabaen . hamare grahak seva pratinidhi se baat karne ke liye , do dabaen . 

['d', 'm', 'i', 'faeneans', 'me', '_', 'ap', 'ka', 'swagat', 'hae', '.', 'lon', 'sambandhit', 'jankari', 'ke', 'liye', 'ek', 'dabaen', '.', 'hamare', 'grahak', 'seva', 'pratinidhi', 'se', 'baat', 'karne', 'ke', 'liye', ',', 'do', 'dabaen', '.', '']

{'', 'me', 'jankari', '_', 'se', 'seva', '.', 'karne', 'd', 'ka', 'swagat', 'liye', 'hamare', 'faeneans', 'i', 'm', 'pratinidhi', 'grahak', 'ek', ',', 'sambandhit', 'ap', 'lon', 'baat', 'ke', 'hae', 'do', 'dabaen'}

28


In [5]:
# Saving the above list to a text file with proper index. 
# f = open('words_2.txt', 'w')

# for index, word in enumerate(text_set):
#     f.write(str(index) + '. ' + word + '\n')
   
# f.close()

# So here '_' means extra a blank space (extra pause), ',' means comma, '.' means period.


In [6]:
# Now importing that word_2.txt to env.
# f = open('words_1.txt', 'r')
# f_content = f.read()
# print(f_content)


In [7]:
# Now we need a long audio file that will contain all the words distinctly. 

# Let's do a fine cropping program based on amplitude. 

# Importing the audio.
ar, sr = librosa.load('data/system_1/dub_3.wav')

print(ar.shape)
print(sr)

Audio(data=ar, rate=sr)


(164787,)
22050


In [8]:
intervals = librosa.effects.split(ar, top_db=18)
print(intervals)
print(len(intervals))

all_words = np.array([])

# Creating arrays for each words
for i in range(len(intervals)):
    inst_interval = intervals[i]
    inst_audio = ar[inst_interval[0] : inst_interval[1]]
    
    # Only use this when you are saving audio clips.
#     sf.write('data/system_2/words/{}.wav'.format(i+16), inst_audio, sr, 'PCM_24')
    
    # Concatinating each words.
    all_words = np.concatenate([all_words, inst_audio])

Audio(all_words, rate=sr)


[[  4096  15872]
 [ 23040  31744]
 [ 37376  48640]
 [ 55808  69632]
 [ 78336  83968]
 [ 90112  97280]
 [103936 112640]
 [118784 124928]
 [127488 135680]
 [143360 149504]
 [153600 163328]]
11


In [9]:
# Now we have all the recording saved.
# Now we need to create some index for the vocabulary.

word_index_dict = {
    'd': 0,
    'm': 1,
    'i': 2,
    'faeneans': 3,
    'me': 4,
    'ap': 5,
    'ka': 6,
    'swagat': 7,
    'hae': 8,
    'lon': 9,
    'sambandhit': 10,
    'jankari': 11,
    'ke': 12,
    'liye': 13,
    'ek': 14,
    'dabaen': 15,
    'hamare': 16,
    'grahak': 17,
    'seva': 18,
    'pratinidhi': 19,
    'se': 20,
    'baat': 21,
    'karne': 22,
    'ke': 23,
    'liye': 24,
    'do': 25,
    'dabaen': 26,
    '_': 27,
    ',': 28,
    '.': 29,
    'ankush': 30
}

# Now we have our word_index dict ready. 


In [10]:
# Let's import all of our recordings. 
file_list = os.listdir('data/system_1/words')
print(len(file_list))
print(file_list)


30
['25.wav', '20.wav', '8.wav', '4.wav', '3.wav', '0.wav', '13.wav', '28.wav', '23.wav', '15.wav', '17.wav', '6.wav', '19.wav', '22.wav', '24.wav', '16.wav', '1.wav', '2.wav', '12.wav', '29.wav', '26.wav', '18.wav', '10.wav', '27.wav', '11.wav', '7.wav', '14.wav', '9.wav', '21.wav', '5.wav']


In [11]:
index_ar_dict = {}

for file_name in file_list:
    inst_arr, inst_sr = librosa.load('data/system_1/words/{}'.format(file_name))
    index_ar_dict[file_name.split('.')[0]] = inst_arr
    
display(index_ar_dict)


{'25': array([ 0.04010546,  0.03652823,  0.03606737, ...,  0.00207925,
         0.00063789, -0.00288546], dtype=float32),
 '20': array([ 0.02827132, -0.0407294 ,  0.02602577, ...,  0.00928533,
         0.0049001 ,  0.00472355], dtype=float32),
 '8': array([-1.4376879e-02, -1.8990517e-02, -4.4703484e-05, ...,
         7.7463627e-02,  7.2459579e-02,  7.6155782e-02], dtype=float32),
 '4': array([-0.00019848, -0.00405216, -0.00016189, ...,  0.00979745,
         0.00540364,  0.00717092], dtype=float32),
 '3': array([-0.03429615, -0.02918947, -0.01673484, ..., -0.01588094,
        -0.0325321 , -0.02219319], dtype=float32),
 '0': array([0.00654626, 0.01052976, 0.01049244, ..., 0.01831615, 0.02720511,
        0.02404392], dtype=float32),
 '13': array([-0.00396478, -0.00701547, -0.00781071, ...,  0.01490629,
         0.01141798,  0.01223028], dtype=float32),
 '28': array([-0.00963746, -0.01808415, -0.01670012, ...,  0.00824768,
         0.00949669,  0.        ], dtype=float32),
 '23': array([ 0

In [12]:
# Let's play a file from the dict
word = index_ar_dict['7']
Audio(word, rate=sr)


In [13]:
%%time
# Let's construct some sentences
# sentence = 'hamare lon seva me ap _ ka swagat hae . . ' \
# 'd m i faeneans se jankari ke liye _ ek _ dabaen . . ' + \
# 'grahak seva pratinidhi me swagat hae . . ' + \
# 'd m i faeneans se _ lon ke liye _ do _ dabaen . ' + \
# 'pratinidhi sambandhit '

sentence = 'd m i faeneans ke grahak seva pratinidhi me ap _ ka swagat hae'


sentence_ar = sentence.split(' ')
print(sentence_ar)

numeric_ar = []

for word in sentence_ar:
    inst_number = word_index_dict[word]
    numeric_ar.append(str(inst_number))
    
print(numeric_ar)



['d', 'm', 'i', 'faeneans', 'ke', 'grahak', 'seva', 'pratinidhi', 'me', 'ap', '_', 'ka', 'swagat', 'hae']
['0', '1', '2', '3', '23', '17', '18', '19', '4', '5', '27', '6', '7', '8']
CPU times: user 364 µs, sys: 101 µs, total: 465 µs
Wall time: 355 µs


In [14]:
%%time
# Re-assembling the words.
final_voice = np.array([])

for number in numeric_ar:
    inst_audio_ar = index_ar_dict[number]
    final_voice = np.concatenate([final_voice, inst_audio_ar])


# Speed shifting
# final_voice = pyrubberband.pyrb.time_stretch(final_voice, sr, 1, rbargs=None)

# # Pitch shifting
# final_voice = pyrubberband.pyrb.pitch_shift(final_voice, sr, 1, rbargs=None)

# # Volume shifting
# final_voice = final_voice - 0
    
# Write the final audio.
sf.write('output_{}.wav'.format(str(1)), final_voice, sr, 'PCM_24')

Audio(final_voice, rate=sr)



CPU times: user 5.42 ms, sys: 3.33 ms, total: 8.75 ms
Wall time: 10.7 ms


## The Major Experiment

In [15]:
# Let's do a fine cropping program based on amplitude. 

# Importing the audio.
ar, sr = librosa.load('data/system_1/dub_1.wav')

print(ar.shape)
print(sr)

Audio(data=ar, rate=sr)


(137984,)
22050


In [16]:
intervals = librosa.effects.split(ar, top_db=8)
print(intervals)

word_1 = np.concatenate([ar[5632 : 18432], ar[22528 : 33792], ar[40960 : 52224], ar[56320 : 69632]])
Audio(word_1, rate=sr)



[[ 10240  11776]
 [ 17920  18432]
 [ 27136  32256]
 [ 44032  53760]
 [ 64000  67072]
 [ 76800  81408]
 [ 92672  94720]
 [108032 113152]
 [114688 117248]
 [130048 131584]]


In [17]:
# Let's find out the number of unique words in i_18 messages. 

f_i_18 = open('data/production/i_18_messages.txt')
content = f_i_18.read()
f_i_18.close()

# Cleaning the contents
content = content.replace(',', '')
content = content.replace('.', '')
content = content.replace('\n', '')
content = content.replace('"', '')
content = content.lower()

# print(content)

i_18_ar = content.split(' ')
# print(i_18_ar)
i_18_set = set(i_18_ar)
# print(i_18_set)
print(len(i_18_ar))
print(len(i_18_set))

time = ((25/20) * 800) / 60 
print(time, 'minutes')

print('status: possible')

6387
750
16.666666666666668 minutes
status: possible


In [18]:
ankush - (syllables) -  
sai 
# jan 6 2022 - 

for name entity  - go with open source for phonemes. 


SyntaxError: invalid syntax (3744854195.py, line 1)